# Prepare CWB Precipitation data

We confirmed that QPESUMS data is in UTC during the analysis process, and hence we need to prepare the precipitation data from CWB stations to meet the time-stamps of QPESUMS data, or the other way around.

## Aggregate the Precipitation Data in UTC



In [1]:
import sys, os, csv, json
import numpy as np
import pandas as pd

DATADIR = '../hr_rain_utc'
flist = os.listdir(DATADIR)
flist.sort()

print(flist)

['466880_rain_hr.txt', '466900_rain_hr.txt', '466910_rain_hr.txt', '466920_rain_hr.txt', '466930_rain_hr.txt', '466940_rain_hr.txt', '466950_rain_hr.txt', 'C0A520_rain_hr.txt', 'C0A530_rain_hr.txt', 'C0A540_rain_hr.txt', 'C0A550_rain_hr.txt', 'C0A560_rain_hr.txt', 'C0A570_rain_hr.txt', 'C0A580_rain_hr.txt', 'C0A640_rain_hr.txt', 'C0A650_rain_hr.txt', 'C0A660_rain_hr.txt', 'C0A710_rain_hr.txt', 'C0A860_rain_hr.txt', 'C0A870_rain_hr.txt', 'C0A880_rain_hr.txt', 'C0A890_rain_hr.txt', 'C0A920_rain_hr.txt', 'C0A930_rain_hr.txt', 'C0A931_rain_hr.txt', 'C0A940_rain_hr.txt', 'C0A950_rain_hr.txt', 'C0A970_rain_hr.txt', 'C0A980_rain_hr.txt', 'C0A9A0_rain_hr.txt', 'C0A9B0_rain_hr.txt', 'C0A9C0_rain_hr.txt', 'C0A9E0_rain_hr.txt', 'C0A9F0_rain_hr.txt', 'C0A9G0_rain_hr.txt', 'C0A9I1_rain_hr.txt', 'C0AC40_rain_hr.txt', 'C0AC60_rain_hr.txt', 'C0AC70_rain_hr.txt', 'C0AC80_rain_hr.txt', 'C0ACA0_rain_hr.txt', 'C0AD00_rain_hr.txt', 'C0AD10_rain_hr.txt', 'C0AD20_rain_hr.txt', 'C0AD30_rain_hr.txt', 'C0AD40_r

In [25]:
# Testmwith one file
data = pd.read_fwf(DATADIR+'/'+flist[1], wisths=[7,11, 10], header=None)
data.columns = ['std_id', 'timestamp', 'prec']
print(data.head())
print(data.shape)

# Filter with time
data = data.loc[(data.timestamp>=2013010100)&(data.timestamp<2017010100),:]
print(data.shape)
print(4*365*24)

# Handle missing data
p = data.prec
perr = p.loc[p<0]
p.loc[p<0] = np.nan

print(np.histogram(perr))

print(np.histogram(p.loc[~np.isnan(p)]))

   std_id   timestamp  prec
0  466900  2000123117   0.0
1  466900  2000123118   0.0
2  466900  2000123119   0.0
3  466900  2000123120   0.0
4  466900  2000123121   0.0
(140256, 3)
(35057, 3)
35040
(array([  0,   0,   0,   0,   0, 975,   0,   0,   0,   0]), array([-999.5, -999.4, -999.3, -999.2, -999.1, -999. , -998.9, -998.8,
       -998.7, -998.6, -998.5]))
(array([33706,   222,    76,    28,    18,    11,     7,     4,     4,
           6]), array([ 0. ,  4.9,  9.8, 14.7, 19.6, 24.5, 29.4, 34.3, 39.2, 44.1, 49. ]))


In [3]:
def process_prec_utc(fname, start_time=2013010100, end_time=2016123123):
    import numpy as np
    import pandas as pd
    # Read in file
    data = pd.read_fwf(fname, wisths=[7,11, 10], header=None)
    data.columns = ['std_id', 'timestamp', 'prec']
    print("Full records: "+ str(data.shape))
    # Cut off required time
    data = data.loc[(data.timestamp>=start_time)&(data.timestamp<=end_time),:]
    print("# records in time range "+str(start_time)+' - ' + str(end_time) +' : '+ str(data.shape))
    # Replace -999, -998, ...etc with numpy.nan
    data.loc[data.prec <0, 'prec'] = np.nan
    print('# missing records: '+ str(sum(np.isnan(data.prec))))
    # Remove the id column
    stdid = data.std_id.iloc[0]
    newdata = pd.DataFrame({'timestamp':data.timestamp, str(stdid):data.prec})
    # Done
    return(newdata)

In [36]:
print('Expected # of records: ' + str(365*4*24+24))
for f in flist:
    tmp = process_prec_utc(DATADIR+'/'+f)

Expected # of records: 35064
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 2774
Full records: (140256, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 975
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 2975
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 2817
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 1910
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 2442
Full records: (140256, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 1258
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 2795
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 

## Check Number of Records per Year

The total number of records don't match, let's do some further check.

In [41]:
ts = list(tmp.timestamp.astype(str))
ts2013 = [t for t in ts if t[0:4]=='2013']
ts2014 = [t for t in ts if t[0:4]=='2014']
ts2015 = [t for t in ts if t[0:4]=='2015']
ts2016 = [t for t in ts if t[0:4]=='2016']

print(len(ts))
print(len(ts2013))
print(len(ts2014))
print(len(ts2015))
print(len(ts2016))

35057
8760
8760
8760
8777


## Number of Records per Month in 2016

It seems the problem is in 2016, let's check month by month.

In [47]:
tsmc = {}
mlab = ['01','02','03','04','05','06','07','08','09','10','11','12']
for m in range(12):
    tsm = [t for t in ts2016 if t[4:6]==mlab[m]]
    #print(tsm)
    tsmc[mlab[m]] = (len(tsm))

print(tsmc)

{'01': 744, '02': 696, '03': 744, '04': 720, '05': 744, '06': 720, '07': 744, '08': 744, '09': 720, '10': 744, '11': 720, '12': 737}


OK, it's in December, and 7 records missing. This is probabily 7 records in the end missing due to the LST-UTC conversion.

In [4]:
#
prec = process_prec_utc(DATADIR+'/'+flist[0])
for f in flist[1:]:
    tmp = process_prec_utc(DATADIR+'/'+f)
    prec = pd.merge(prec, tmp, on='timestamp')
    
prec.shape

Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 2774
Full records: (140256, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 975
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 2975
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 2817
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 1910
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 2442
Full records: (140256, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 1258
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 2795
Full records: (324360, 3)
# records in time range 2013010100 - 2016123123 : (35057, 3)
# missing records: 933
Ful

(35057, 51)

In [55]:
prec.head()

,timestamp,466880,466900,466910,466920,466930,466940,466950,C0A520,C0A530,...,C0ACA0,C0AD00,C0AD10,C0AD20,C0AD30,C0AD40,C0AD50,C0AG90,C0AH00,C0AH10
0,2013010100,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2013010101,NaN,0.0,NaN,NaN,0.0,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2013010102,NaN,0.0,NaN,0.5,NaN,1.0,0.0,0.0,0.5,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,NaN
3,2013010103,NaN,0.0,0.5,NaN,0.5,0.5,0.0,0.5,0.5,...,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.5,0.0,NaN
4,2013010104,1.5,0.0,1.5,1.5,0.5,1.0,0.1,0.5,1.0,...,1.0,0.0,0.0,1.0,0.5,1.0,0.5,2.0,1.5,NaN


In [5]:
prec.to_csv('data/t1hr_utc.csv', index=False)